In [1]:
import os
import glob
import pandas as pd
import numpy as np

In [ ]:
metrics = ['seed', 't_training_avg', 't_training_std',
            'RMSEValidation','EnergyScoreValidation', 'CRPSValidation',
            'Gaussian NLLValidation', 'CoverageValidation']

path = "../results/KS/hparams/beta/mixednormal/" #"../results/Burgers/hparams/mvnormal/"
file_list = ([f for f in glob.iglob(path + "**", recursive = True) if os.path.isfile(f) and f.endswith("test.csv")])
ids = ["beta_endpoints"]
for id in ids:
    metrics.append(id)
factor = 10

In [3]:
results = pd.DataFrame()
for file in file_list:
    if os.path.exists(file):
        results_df = pd.read_csv(file, index_col=0)
        results = pd.concat([results, results_df], axis = 1)
rows = metrics.copy()
results = results.loc[rows]
metrics.pop()
results.loc[metrics] = results.loc[metrics].astype("float32")
results = results.transpose()
results.index = results[id]
results.drop(id, axis = 1, inplace = True)
results.sort_values(ids, inplace = True)
results.insert(0, "beta",[0.2,0.2,0.35,0.35])
results["beta"] = [0.2,0.2,0.35,0.35]

In [4]:
results

,beta,seed,t_training_avg,t_training_std,RMSEValidation,EnergyScoreValidation,CRPSValidation,Gaussian NLLValidation,CoverageValidation
beta_endpoints,,,,,,,,,
"(0.001, 0.2)",0.20,12345.0,10.515783,0.088675,0.004909,0.065604,0.00257,-3.807157,0.999566
"[0.001, 0.2]",0.20,1234.0,10.493962,0.083198,0.004583,0.06336,0.002594,-3.827182,0.999722
"[0.001, 0.35]",0.35,1234.0,10.599971,0.226976,0.006264,0.083255,0.003415,-3.560698,0.999596
"[0.001, 0.35]",0.35,12345.0,10.515601,0.087133,0.004858,0.063456,0.002589,-3.803216,0.999431


In [5]:
results = results.groupby(by = ["beta"]).mean().drop(["seed", "t_training_avg", "t_training_std"], axis = 1)
results.iloc[:,0:3] *= factor
results

,RMSEValidation,EnergyScoreValidation,CRPSValidation,Gaussian NLLValidation,CoverageValidation
beta,,,,,
0.20,0.047458,0.644821,0.025816,-3.81717,0.999644
0.35,0.055613,0.733557,0.030022,-3.681957,0.999514


In [6]:
latex_table = results.to_latex(escape=False, float_format="%.4f")
print(latex_table)

\begin{tabular}{llllll}
\toprule
 & RMSEValidation & EnergyScoreValidation & CRPSValidation & Gaussian NLLValidation & CoverageValidation \\
beta &  &  &  &  &  \\
\midrule
0.200000 & 0.0475 & 0.6448 & 0.0258 & -3.8172 & 0.9996 \\
0.350000 & 0.0556 & 0.7336 & 0.0300 & -3.6820 & 0.9995 \\
\bottomrule
\end{tabular}



# Runtime

In [2]:
metrics = ['distributional_method', 't_training_avg', 't_training_std',]

exp = "KS"
path = f"../results/timing/{exp}/" #"../results/Burgers/hparams/mvnormal/"
file_list = ([f for f in glob.iglob(path + "**", recursive = True) if os.path.isfile(f) and f.endswith("csv")])

In [3]:
results = pd.DataFrame()
for file in file_list:
    if os.path.exists(file):
        results_df = pd.read_csv(file, index_col=0)
        results = pd.concat([results, results_df], axis = 1)
rows = metrics.copy()
results = results.loc[rows].T

# results.loc[metrics] = results.loc[metrics].astype("float32")
# results = results.transpose()
# results.index = results[id]
# results.drop(id, axis = 1, inplace = True)
# results.sort_values(ids, inplace = True)
# results.insert(0, "beta",[0.2,0.2,0.35,0.35])
# results["beta"] = [0.2,0.2,0.35,0.35]

In [4]:
results["t_training_avg"] = results["t_training_avg"].astype("float")
results["t_training_std"] = results["t_training_std"].astype("float")

In [5]:
results

,distributional_method,t_training_avg,t_training_std
0,deterministic,7.148298,0.134955
1,deterministic,6.942949,0.077589
2,deterministic,6.550388,0.198142
3,deterministic,6.459361,0.049449
4,deterministic,6.468215,0.050431
0,normal,6.702718,0.119630
1,normal,6.472809,0.068338
2,normal,7.090417,0.630314
3,normal,7.108717,0.493425
4,normal,7.389230,0.183316


In [6]:
mean = results.groupby("distributional_method").mean()
var = results.groupby("distributional_method").var()

In [7]:
# Mean of variances
E_s2 = results.groupby("distributional_method")["t_training_std"].apply(lambda s: (s**2).mean())
# Variance of  means
Var_m = var["t_training_avg"]
# Pure mean
final_mean = mean["t_training_avg"]

final_std = np.sqrt(E_s2 + Var_m)

results = pd.DataFrame({
    "final_mean": final_mean,
    "final_std": final_std
}).T

In [8]:
# construct table with makecell entries
latex_df = pd.DataFrame(index=[" "])  # single-row table

for col in results.columns:
    mean = results.loc["final_mean", col]
    std  = results.loc["final_std", col]

    latex_df[col] = (
        rf"\makecell{{{mean:.2f} \\ ($\pm$ {std:.2f})}}"
    )

# produce LaTeX table
latex_table = latex_df.to_latex(escape=False)
print(latex_table)

\begin{tabular}{llllll}
\toprule
 & deterministic & mixednormal & mvnormal & normal & sample \\
\midrule
  & \makecell{6.71 \\ ($\pm$ 0.33)} & \makecell{10.37 \\ ($\pm$ 0.08)} & \makecell{6.90 \\ ($\pm$ 0.12)} & \makecell{6.95 \\ ($\pm$ 0.52)} & \makecell{10.41 \\ ($\pm$ 0.04)} \\
\bottomrule
\end{tabular}

